# RAG-система
На основе статьи Deep learning for algorithmic trading: A systematic review of predictive models and optimization strategies

https://www.sciencedirect.com/science/article/pii/S2590005625000177

- Индексирует текст статьи (разбивает на фрагменты, создает векторные представления).
- Извлекает релевантные фрагменты по запросу пользователя.
- Генерирует ответ, используя и контекст из статьи, и знания языковой модели.

In [1]:
%%capture --no-stderr

install = False
if install:
  !pip install langchain-text-splitters langchain_community langchain_huggingface
  !pip install langfuse
  !pip install faiss-cpu

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Импорты
import os
import time
import torch
import psutil
from typing import List, Dict, Any

In [4]:
# ========== 1. НАСТРОЙКА LANGFUSE ==========
from google.colab import userdata

os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('LANGFUSE_PUBLIC_KEY')
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get('LANGFUSE_SECRET_KEY')
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

from langfuse import Langfuse
from langfuse.langchain import CallbackHandler

langfuse_client = Langfuse()
langfuse_handler = CallbackHandler()

In [5]:
# Проверка доступности объектов Langfuse
print("Проверка объектов Langfuse:")
print(f"langfuse_client существует: { 'langfuse_client' in globals() }")
print(f"langfuse_handler существует: { 'langfuse_handler' in globals() }")
print(f"Тип клиента: {type(langfuse_client) if 'langfuse_client' in globals() else 'НЕТ'}")
print(f"Тип хэндлера: {type(langfuse_handler) if 'langfuse_handler' in globals() else 'НЕТ'}")

# Проверка подключения к серверу
try:
    auth = langfuse_client.auth_check()
    print(f"\n✅ Подключение к Langfuse: {auth} ")
except Exception as e:
    print(f"\n⚠️ Ошибка подключения: {e}")
    print("   Проверьте ключи и хост в переменных окружения!")

Проверка объектов Langfuse:
langfuse_client существует: True
langfuse_handler существует: True
Тип клиента: <class 'langfuse._client.client.Langfuse'>
Тип хэндлера: <class 'langfuse.langchain.CallbackHandler.LangchainCallbackHandler'>

✅ Подключение к Langfuse: True 


In [6]:
# ========== 2. ПРОВЕРКА РЕСУРСОВ COLAB ==========
print("=== Проверка ресурсов Colab ===")
if torch.cuda.is_available():
    gpu_props = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu_props.name}")
    print(f"Память: {gpu_props.total_memory / 1024**3:.2f} GB")
    # Qwen2.5-3B в float16 требует ~6 ГБ — проверяем доступность
    if gpu_props.total_memory < 7 * 1024**3:
        print("⚠️ Внимание: маловато памяти GPU. Рассмотрите Qwen2.5-1.5B или использование CPU.")
else:
    print("⚠️ GPU недоступен — будет использоваться CPU (медленно!)")

=== Проверка ресурсов Colab ===
GPU: Tesla T4
Память: 14.74 GB


In [7]:
# ========== 3. ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ==========
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
loader = TextLoader('/content/drive/MyDrive/01_LLM/OTUS/HW5_article.txt', encoding='utf-8')
documents = loader.load()
chunks = text_splitter.split_documents(documents)

print(f"✅ Создано {len(chunks)} чанков")

✅ Создано 319 чанков


In [8]:
# ========== 4. ВЕКТОРНАЯ БАЗА ==========
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
)

vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print("✅ Векторная база создана")

✅ Векторная база создана


In [9]:
# ========== 5. ЗАГРУЗКА QWEN2.5-3B-INSTRUCT ==========
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

model_name = "Qwen/Qwen2.5-3B-Instruct"

print(f"Загрузка модели {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True  # Экономия памяти при загрузке
)

text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
    do_sample=True,
    return_full_text=False,
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
print("✅ Модель загружена")


Загрузка модели Qwen/Qwen2.5-3B-Instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Модель загружена


In [10]:
# ========== 6. RAG ЦЕПЬ С ПОЛНЫМ МОНИТОРИНГОМ ==========
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Форматирование для Qwen (специфичный шаблон)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты — эксперт по Deep learning for algorithmic trading. Отвечай точно по контексту."),
    ("user", "Контекст:\n{context}\n\nВопрос: {question}\n\nОтвет:")
])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
# ========== 7. ФУНКЦИЯ ЗАПРОСА С МЕТРИКАМИ ==========
import time
import psutil
from typing import Dict, Any

def ask_question(question: str) -> Dict[str, Any]:
    """RAG-запрос с автоматическим трекингом через Langfuse"""

    mem_before = psutil.Process().memory_info().rss / 1024**2

    # === ШАГ 1: Извлечение документов ===
    start = time.time()
    docs = retriever.invoke(question)
    retrieval_time = time.time() - start

    # === ШАГ 2: Генерация ответа ===
    start = time.time()

    # Точный подсчёт токенов
    context_text = "\n\n".join(doc.page_content for doc in docs)
    full_prompt = f"Контекст:\n{context_text}\n\nВопрос: {question}\n\nОтвет:"
    input_tokens = len(tokenizer.encode(full_prompt))

    # Автоматический трекинг через колбэк
    answer = rag_chain.invoke(
        question,
        config={"callbacks": [langfuse_handler]}  # ← автоматически создаёт трейс
    )

    generation_time = time.time() - start
    output_tokens = len(tokenizer.encode(answer))
    total_latency = retrieval_time + generation_time

    mem_after = psutil.Process().memory_info().rss / 1024**2

    # === ВЫВОД РЕЗУЛЬТАТОВ ===
    print(f"\n{'='*70}")
    print(f"❓ ВОПРОС: {question}")
    print(f"{'='*70}")
    print(f"\n💡 ОТВЕТ:\n{answer}")
    print(f"\n📊 МЕТРИКИ:")
    print(f"   • Общая задержка: {total_latency:.2f} с")
    print(f"   • Поиск: {retrieval_time:.2f} с | Генерация: {generation_time:.2f} с")
    print(f"   • Токены: вход={input_tokens}, выход={output_tokens}, всего={input_tokens+output_tokens}")
    print(f"   • Память: +{mem_after - mem_before:.1f} MB")
    print(f"\n📄 ИСТОЧНИКИ ({len(docs)}):")
    for i, doc in enumerate(docs[:2], 1):
        print(f"   {i}. {doc.page_content[:150]}...")
    print(f"{'='*70}\n")

    langfuse_client.flush()

    return {
        "answer": answer,
        "sources": docs,
        "metrics": {
            "total_latency": total_latency,
            "retrieval_latency": retrieval_time,
            "generation_latency": generation_time,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "memory_used_mb": mem_after - mem_before
        }
    }

In [12]:
print("Тестовая демонстрация...")
result = ask_question("Каковы последние достижения в применении алгоритмов глубокого обучения к алгоритмической торговле?")
print("\n✅ Тест пройден успешно!")
print(f"Средняя задержка: {result['metrics']['total_latency']:.2f} с")
print(f"Токены: {result['metrics']['input_tokens'] + result['metrics']['output_tokens']}")
print("\n🔗 Трейсы появятся в Langfuse через 5-10 секунд:")
print("   https://cloud.langfuse.com")

Тестовая демонстрация...

❓ ВОПРОС: Каковы последние достижения в применении алгоритмов глубокого обучения к алгоритмической торговле?

💡 ОТВЕТ:
 Последние достижения в области применения алгоритмов глубокого обучения для алгоритмической торговли значительно обогатили возможности автоматизации и оптимизации стратегий на биржевых рынках. Вот несколько ключевых направлений:

1. Машинное обучение и прогнозирование курса акций: Исследования показывают значительный прогресс в использовании нейронных сетей для предсказания движения цен на акции. Например, методы LSTM (Long Short-Term Memory) используются для анализа временных рядов данных и выявления тенденций.

2. Сентиментальный анализ и макроэкономика: Глубокое обучение активно применяется для анализа текстовых сообщений о компании с помощью техник такие как Word Embeddings или Transformer-based модели. Это позволяет улучшить точность прогнозирования цены на основе общего аудита публичной информации.

3. Агрегация различных источников дан

In [13]:
print("="*80)
print("✨ ДЕМОНСТРАЦИЯ RAG-СИСТЕМЫ С МОНИТОРИНГОМ LANGFUSE ✨")
print("="*80)
print(f"Модель: Qwen2.5-3B-Instruct | Чанков: {len(chunks)} | GPU: {torch.cuda.is_available()}")
print("="*80 + "\n")

demo_questions = [
    ("📊 Архитектуры", "Какие архитектуры глубокого обучения применяются в алгоритмической торговле?"),
    ("⚠️ Проблемы", "Какие основные проблемы возникают при применении deep learning в трейдинге?"),
    ("📈 Оценка", "Как оценивается эффективность моделей в обзоре?")
]

all_metrics = []
for label, question in demo_questions:
    print(f"\n{'─'*80}")
    print(f"🔹 {label}")
    print(f"❓ Вопрос: {question}")
    print('─'*80)

    result = ask_question(question)
    all_metrics.append(result['metrics'])
    time.sleep(1.5)

# Сводная таблица
print("\n" + "="*80)
print("📈 СВОДНАЯ ТАБЛИЦА МЕТРИК")
print("="*80)
print(f"{'№':<4} {'Задержка (с)':<15} {'Токены':<20} {'Память (МБ)':<15}")
print("-"*80)
for i, m in enumerate(all_metrics, 1):
    total_tokens = m['input_tokens'] + m['output_tokens']
    print(f"{i:<4} {m['total_latency']:<15.2f} {total_tokens:<20} {m['memory_used_mb']:<15.1f}")
print("="*80)
print(f"✅ Средняя задержка: {sum(m['total_latency'] for m in all_metrics)/len(all_metrics):.2f} с")
print(f"✅ Всего токенов: {sum(m['input_tokens']+m['output_tokens'] for m in all_metrics)}")
print(f"\n🔗 Трейсы доступны в Langfuse: https://cloud.langfuse.com")
print("="*80)

langfuse_client.flush()
print("\n✅ Все данные отправлены в Langfuse!")

✨ ДЕМОНСТРАЦИЯ RAG-СИСТЕМЫ С МОНИТОРИНГОМ LANGFUSE ✨
Модель: Qwen2.5-3B-Instruct | Чанков: 319 | GPU: True


────────────────────────────────────────────────────────────────────────────────
🔹 📊 Архитектуры
❓ Вопрос: Какие архитектуры глубокого обучения применяются в алгоритмической торговле?
────────────────────────────────────────────────────────────────────────────────

❓ ВОПРОС: Какие архитектуры глубокого обучения применяются в алгоритмической торговле?

💡 ОТВЕТ:
 В алгоритмической торговле для анализа и прогнозирования цен на акции часто используются различные архитектуры глубокого обучения. Вот некоторые из наиболее популярных:

1. **Рекуррентные нейронные сети (RNN)**:
   - Рекурсивные модели хорошо подходят для обработки последовательностей данных, таких как исторические цены акций или текстовые данные из социальных сетей.

2. **Слайдерские рекуррентные нейронные сети (SL-RNN)**:
   - Улучшенная версия RNN, которая позволяет эффективнее обрабатывать длинные временные ряды с пом

Отчёт по реализации и тестированию RAG-системы с мониторингом через Langfuse: https://docs.google.com/document/d/1OlArZ_inpa4g7oU-1CubvN_rzLXdeBdru-JW3q8bljk/edit?usp=sharing